In [ ]:
import json
import os
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
import LAC
import happybase
import joblib
import sklearn.utils

In [ ]:
df_list = []
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/pre_age_200910_train_res") as f:
    for line in f:
        df_list.append(line.strip().split('\001'))
df = pd.DataFrame(df_list)

In [ ]:
df_columns_list = ["udid","age"]
for i in range(10):
    df_columns_list.append("c{}".format(i))
df.columns = df_columns_list

In [ ]:
df

In [ ]:
aid_set = set()
for i in range(10):
    aid_set.update(df["c{}".format(i)])

In [ ]:
a_dict = {}
with open("/mnt1/zhaodachuan/data/predict_user_attribute20200911/raw_data/user_profile/title_res.jsonl") as f:
    for line in f:
        temp_dict = json.loads(line.strip())
        aid = temp_dict["aid"]
        title_tokenize = temp_dict["title_tokenize"]
        a_dict[aid] = title_tokenize

In [ ]:
def temp_function(x):
    #x = datetime.datetime.now().year - int(x.split("-")[0])
    y = 0
    if x < 20:
        y = "~20"
    elif x<25:
        y = "21~25"
    elif x<29:
        y = "26~29"
    elif x<35:
        y = "30~35"
    elif x<45:
        y = "36~45"
    elif x<50:
        y = "46~50"
    else:
        y = "50~"
    return y

In [ ]:
title_tokenize_list = []
age_list = []
for index in df.index:
    age = temp_function(int(df.loc[index,"age"]))
    temp_list = []
    for i in range(10):
        c = "c{}".format(i)
        if str(df.loc[index,c]):
            if str(df.loc[index,c]) in a_dict:
                temp_list.extend(a_dict[str(df.loc[index,c])])
                temp_list.append("<PADDING>")
            else:
                print("aid不存在",index,c,df.loc[index,c])
        else:
            print(index)
    age_list.append(age)
    title_tokenize_list.append(temp_list)

In [ ]:
with open("/home/zhaodachuan/jupyter_lab/user_profile/age/data/最终数据集tokenize.jsonl","w") as f:
    for tag,content_tokenize in zip(age_list,title_tokenize_list):
        f.write(json.dumps({"tag":tag,"content_tokenize":content_tokenize},ensure_ascii=False) + "\n")